In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# ref : https://www.kaggle.com/robikscube/nfl-1st-and-future-analytics-intro
# For tracking the player route

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# Distribution of surface natural vs syntetic
playlist = pd.read_csv('/kaggle/input/nfl-playing-surface-analytics/PlayList.csv')
#playlist.FieldType



In [ ]:
# check the PlayerTrack of player
playerTrack = pd.read_csv('/kaggle/input/nfl-playing-surface-analytics/PlayerTrackData.csv')


In [ ]:
# What is percentage  of playing in natural vs synthetic turf
# natural surface percentage is 59.87%  (149723 is 59.87% of 250095)
# synthetic surface percentage is 40.13%  (100372 is 40.13% of 250095)

In [ ]:
playlist.groupby('FieldType').count()['StadiumType'].sum()
playlist.groupby('StadiumType')

In [ ]:
import matplotlib.pylab  as plt
import seaborn as sns
sns.set_style('whitegrid')
playlist.groupby('FieldType').count() \
            .sort_values('StadiumType') \
            .plot(kind ='bar', figsize=(15,5), title='distribution of natural surf vs sythetic')
plt.show()


In [ ]:
# check the InjuryRecord of player
injlist = pd.read_csv('/kaggle/input/nfl-playing-surface-analytics/InjuryRecord.csv')

In [ ]:
# plot the distibution of injury occured on bodypart on a given surface
injlist.groupby(['BodyPart','Surface']) \
            .count() \
            .unstack('BodyPart')['PlayKey'] \
            .T.sort_values('Natural').T \
            .sort_values('Surface') \
            .plot(kind ='bar', figsize=(15,5), title='Distibution of injury occured on bodypart on a given surface')
plt.show()


In [ ]:
injlist

In [ ]:
# As you can see from above plot.
# 01. Occurance of  knee injury for a player will be same wether he plays on natural or synthetic turf
# 02. Occurance of  Ankel  injury for a player will be less on Natural turf as compared to synthetic turf 
# 03. Occurance of Foot injury for a player will be More on Natural turf as compared to synthetic turf 

In [ ]:
injlist.groupby('PlayerKey').nunique()

In [ ]:
inj_player = injlist.merge(playlist)

In [ ]:
inj_player.groupby(['RosterPosition','Surface']) \
            .count()['PlayerKey'] \
            .sort_values() \
            .plot(kind ='bar', figsize=(15,5), title='Injury occured at a given position by surface')
plt.show()

In [ ]:
# As you can see from above plot.
# 01. Occurance of  injury at linebacker position is more on natural or synthetic turf
# 02. Followed by Occurance of  injury at Wide position is more on synthetic turf than natural turf


In [ ]:
inj_player

In [ ]:
inj_player.groupby('Surface').count()['PlayerKey'] \


In [ ]:
inj_player.groupby(['RosterPosition','BodyPart']) \
            .count() \
            .unstack('BodyPart')['PlayerKey'] \
            .T.apply(lambda x: x / x.sum()) \
            .sort_values('BodyPart').T.sort_values('Ankle', ascending=False) \
            .plot(kind ='bar', figsize=(15,5), title='Injury occured on a bodypart at a position')
plt.show()

In [ ]:
# As you can see from above plot.
# 01. Occurance of Ankle njury at a position Comeback, Knee injurya while Running Back is more
# 02. Followed by Occurance of  Ankle injury at offensive Line man, Wide position, linebscker is more, Occurane of Knee injury at position safet and line back are equal


In [ ]:
import matplotlib.patches as patches
# ref https://www.kaggle.com/robikscube/nfl-big-data-bowl-plotting-player-position
def create_football_field(linenumbers=True,
                          endzones=True,
                          highlight_line=False,
                          highlight_line_number=50,
                          highlighted_name='Line of Scrimmage',
                          fifty_is_los=False,
                          figsize=(12, 6.33)):
    """
    Function that plots the football field for viewing plays.
    Allows for showing or hiding endzones.
    """
    rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1,
                             edgecolor='r', facecolor='darkgreen', zorder=0)

    fig, ax = plt.subplots(1, figsize=figsize)
    ax.add_patch(rect)

    plt.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
              53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
             color='white')
    if fifty_is_los:
        plt.plot([60, 60], [0, 53.3], color='gold')
        plt.text(62, 50, '<- Player Yardline at Snap', color='gold')
    # Endzones
    if endzones:
        ez1 = patches.Rectangle((0, 0), 10, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ez2 = patches.Rectangle((110, 0), 120, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ax.add_patch(ez1)
        ax.add_patch(ez2)
    plt.xlim(0, 120)
    plt.ylim(-5, 58.3)
    plt.axis('off')
    if linenumbers:
        for x in range(20, 110, 10):
            numb = x
            if x > 50:
                numb = 120 - x
            plt.text(x, 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white')
            plt.text(x - 0.95, 53.3 - 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white', rotation=180)
    if endzones:
        hash_range = range(11, 110)
    else:
        hash_range = range(1, 120)

    for x in hash_range:
        ax.plot([x, x], [0.4, 0.7], color='white')
        ax.plot([x, x], [53.0, 52.5], color='white')
        ax.plot([x, x], [22.91, 23.57], color='white')
        ax.plot([x, x], [29.73, 30.39], color='white')

    if highlight_line:
        hl = highlight_line_number + 10
        plt.plot([hl, hl], [0, 53.3], color='yellow')
        plt.text(hl + 2, 50, '<- {}'.format(highlighted_name),
                 color='yellow')
    return fig, ax

In [ ]:
# Loop through all 99 inj plays
inj_play_list = injlist['PlayKey'].tolist()
fig, ax = create_football_field()
for playkey, inj_play in playerTrack.query('PlayKey in @inj_play_list').groupby(['time']):
    inj_play.plot(kind='scatter', x='x', y='y', ax=ax, color='orange', alpha=0.2)
plt.show()

In [ ]:
playerTrack

In [ ]:
import matplotlib.pylab  as plt
playerTrack.query('PlayKey in @inj_play_list')['s'].plot(kind='hist',
                                                 title='Distribution of player Speed injured',
                                                 figsize=(15, 5), bins=30)
